In [1]:
import open3d as o3d
import numpy as np
import os
import cv2
import copy

storage_path = "data/"
isRunning = True

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def key_callback(vis, action, mods):
    global isRunning
    if action == 0:
        isRunning = False

In [3]:
# 首先尝试从data中打开文件
# 复现点云
filename_str = "dormitory"  # lab_table lab_box dormitory
pt_data = np.load(f"../data/{filename_str}.npz")
points = pt_data["points"]
colors = pt_data["colors"]
vis = o3d.visualization.VisualizerWithKeyCallback()
vis.create_window()
vis.register_key_action_callback(81, key_callback)
pcd = o3d.geometry.PointCloud()
coordinateFrame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=1000, origin=[0, 0, 0]
)
vis.add_geometry(coordinateFrame)

pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)
vis.add_geometry(pcd)
vis.poll_events()
vis.update_renderer()
while isRunning:
    key = cv2.waitKey(1)
    vis.update_geometry(pcd)
    if key == ord("q"):
        break

    vis.poll_events()
    vis.update_renderer()


vis.destroy_window()

In [4]:
print("3D Points Cloud shape: ", points.shape)
print("Color img shape: ", colors.shape)
colors[1]

3D Points Cloud shape:  (230400, 3)
Color img shape:  (230400, 3)


array([0.85490196, 0.89411765, 0.9254902 ])

In [5]:
# 尝试生成原来格式的图片
Height = 40 * 9
Width = 40 * 16
channels = 3
assert Height * Width == points.shape[0]

rescale_colors = 255 * colors
rescale_colors = rescale_colors.astype(np.uint8).reshape((Height, Width, channels))
rescale_colors = cv2.cvtColor(rescale_colors, cv2.COLOR_RGB2BGR)
cv2.imshow("Image", rescale_colors)
cv2.waitKey(0)
cv2.destroyAllWindows()
# 因此这个图片可以用来做基于AI模型的分割，然后思考出一个算法（或者借鉴一个算法）映射到PointCloud的上面
# 发现1：应当把距离过于大的点给滤去，否则无法正常显示;同时，应该把图片中对应的点给抹去
# 否则只能在极好的场景下做测试

In [ ]:
# 使用open3d的接口将pointcloud文件直接转化为基于mesh的obj文件提供三种方法： BPA PSR Alpha-Shape
method_idx = 1  # 1,2,3 for different method

pcd_test = copy.deepcopy(pcd)  # 深拷贝
pcd_test.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=30)
)
# 可以使用 pcd_tets.orient_normals_consistent_tangent_plane(k=10) 来尝试统一法线方向
# 或者 pcd_test.orient_normals_towards_camera_location(camera_location=np.array([0.,0.,0.]))

# 表面重建 (选择一种算法)
# 注意：Poisson重建可能会在点云的凸包区域创建表面，即使那里没有点。
# width 参数可以用来控制这种行为，较低的width值会使重建更紧密地贴合点云。
try:
    mesh_poisson, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd_test, depth=8, width=0, scale=1.1, linear_fit=False
    )
    # 可以选择基于密度去除低置信度的三角面片
    # print("Removing low density triangles...")
    # vertices_to_remove = densities < np.quantile(densities, 0.01) # 移除密度最低1%的顶点相关的面片
    # mesh_poisson.remove_vertices_by_mask(vertices_to_remove)
    if mesh_poisson.has_triangles():
        print("Poisson mesh created successfully.")
        mesh_to_save = mesh_poisson
        # Poisson重建的mesh通常是水密的，但可能需要裁剪掉远离点云的部分。
        # 这里可以添加一个裁剪步骤，例如使用点云的包围盒。
        # bbox = pcd.get_axis_aligned_bounding_box()
        # mesh_to_save = mesh_poisson.crop(bbox)

    else:
        print("Poisson failed to create triangles.")
        if "mesh_to_save" not in locals() or mesh_to_save is None:  # 如果BPA也没成功
            mesh_to_save = None
except Exception as e:
    print(f"Poisson reconstruction failed: {e}")
    if "mesh_to_save" not in locals() or mesh_to_save is None:  # 如果BPA也没成功
        mesh_to_save = None

# 保存
if (
    "mesh_to_save" in locals()
    and mesh_to_save is not None
    and mesh_to_save.has_triangles()
):
    # 可选: 对Mesh进行后处理，如平滑、简化等
    # mesh_to_save = mesh_to_save.filter_smooth_simple(number_of_iterations=5)
    # mesh_to_save = mesh_to_save.simplify_quadric_decimation(target_number_of_triangles=max(1, len(mesh_to_save.triangles) // 2))

    # 4. 保存为.obj文件
    output_obj_file = "reconstructed_mesh.obj"
    print(f"Saving mesh to {output_obj_file}...")
    # write_triangle_mesh 函数可以保存多种格式，通过文件扩展名判断
    # 它会自动处理顶点、法线（如果计算了）和面。
    # 如果点云有颜色，并且你想在mesh中保留顶点颜色，需要确保mesh在重建时也保留了颜色信息。
    # BPA 和 Alpha Shapes 通常能直接从输入点云继承颜色（如果法线估计后颜色还在）。
    # Poisson重建的mesh可能不会直接继承颜色，你可能需要将颜色从原始pcd传递给生成的mesh顶点。
    # if pcd.has_colors() and mesh_to_save.has_vertices():
    #     # 简单的颜色传递方法：基于最近邻顶点赋予颜色
    #     # 注意：这仅在mesh顶点与原始点云顶点有良好对应时效果较好
    #     print("Attempting to color mesh based on original point cloud...")
    #     kdtree = o3d.geometry.KDTreeFlann(pcd)
    #     mesh_vertex_colors = np.zeros_like(np.asarray(mesh_to_save.vertices))
    #     for i, vert in enumerate(mesh_to_save.vertices):
    #         [_, idx, _] = kdtree.search_knn_vector_3d(vert, 1)
    #         mesh_vertex_colors[i] = pcd.colors[idx[0]]
    #     mesh_to_save.vertex_colors = o3d.utility.Vector3dVector(mesh_vertex_colors)

    success = o3d.io.write_triangle_mesh(
        output_obj_file,
        mesh_to_save,
        write_vertex_normals=True,
        write_vertex_colors=False,
    )
    if success:
        print(f"Mesh saved successfully to {output_obj_file}")
    else:
        print(f"Failed to save mesh to {output_obj_file}")

Poisson mesh created successfully.
Saving mesh to reconstructed_mesh.obj...
Mesh saved successfully to reconstructed_mesh.obj
